In [1]:
import pandas as pd
import numpy as np
import sklearn as sns
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("market_analysis_in_banking_domain.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("BankMarketingAnalysis").getOrCreate()

C:\Users\user\Anaconda3\lib\site-packages\pyspark\context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [7]:
df = spark.read.csv("F:\BIG DATA\market_analysis_in_banking_domain.csv", header=True, inferSchema=True)

In [9]:
df.head()

Row(age=58, job='management', marital='married', education='tertiary', default='no', balance=2143, housing='yes', loan='no', contact='unknown', day=5, month='may', duration=261, campaign=1, pdays=-1, previous=0, poutcome='unknown', y='no')

In [10]:
total_count_of_customers = df.count()

In [11]:
total_count_of_customers

45211

In [12]:
total_subscriptions = df.filter(df.y == 'yes').count()
total_not_subscribed = df.filter(df.y == 'no').count()

In [13]:
total_subscriptions

5289

In [14]:
total_not_subscribed

39922

In [15]:
success_rate = (total_subscriptions / total_count_of_customers) * 100

In [16]:
failure_rate = (total_not_subscribed / total_count_of_customers) * 100

In [17]:
print("Marketing Success Rate: {}%".format(success_rate))
print("Marketing Failure Rate: {}%".format(failure_rate))

Marketing Success Rate: 11.698480458295547%
Marketing Failure Rate: 88.30151954170445%


In [18]:
max_age = df.agg({"age": "max"}).collect()[0][0]
mean_age = df.agg({"age": "mean"}).collect()[0][0]
min_age = df.agg({"age": "min"}).collect()[0][0]

In [19]:
print("Maximum Age: {}".format(max_age))
print("Mean Age: {}".format(mean_age))
print("Minimum Age: {}".format(min_age))

Maximum Age: 95
Mean Age: 40.93621021432837
Minimum Age: 18


In [20]:
average_balance = df.agg({"balance": "avg"}).collect()[0][0]
median_balance = df.approxQuantile("balance", [0.5], 0.25)[0]

In [21]:
print("Average Balance: {}".format(average_balance))
print("Median Balance: {}".format(median_balance))

Average Balance: 1362.2720576850766
Median Balance: 72.0


In [22]:
age_subscribed = df.filter(df.y == 'yes').groupBy("age").count().orderBy("age").show()

+---+-----+
|age|count|
+---+-----+
| 18|    7|
| 19|   11|
| 20|   15|
| 21|   22|
| 22|   40|
| 23|   44|
| 24|   68|
| 25|  113|
| 26|  134|
| 27|  141|
| 28|  162|
| 29|  171|
| 30|  217|
| 31|  206|
| 32|  221|
| 33|  210|
| 34|  198|
| 35|  209|
| 36|  195|
| 37|  170|
+---+-----+
only showing top 20 rows



In [23]:
marital_subscribed = df.filter(df.y == 'yes').groupBy("marital").count().show()

+--------+-----+
| marital|count|
+--------+-----+
|divorced|  622|
| married| 2755|
|  single| 1912|
+--------+-----+



In [24]:
age_marital_subscribed = df.filter(df.y == 'yes').groupBy("age", "marital").count().sort("age").show()

+---+--------+-----+
|age| marital|count|
+---+--------+-----+
| 18|  single|    7|
| 19|  single|   11|
| 20|  single|   14|
| 20| married|    1|
| 21|  single|   21|
| 21| married|    1|
| 22|  single|   40|
| 23|  single|   42|
| 23| married|    2|
| 24|  single|   58|
| 24| married|   10|
| 25| married|   14|
| 25|  single|   99|
| 26|  single|  121|
| 26| married|   13|
| 27|divorced|    2|
| 27| married|   29|
| 27|  single|  110|
| 28| married|   20|
| 28|  single|  138|
+---+--------+-----+
only showing top 20 rows



In [25]:
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import StringIndexer

In [26]:
bucketizer = Bucketizer(splits=[0, 30, 40, 50, float('inf')], inputCol="age", outputCol="age_group")

In [27]:
print(bucketizer)

Bucketizer_ebdb53e48305


In [28]:
df = bucketizer.transform(df)

In [29]:
df

DataFrame[age: int, job: string, marital: string, education: string, default: string, balance: int, housing: string, loan: string, contact: string, day: int, month: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, y: string, age_group: double]

In [30]:
df.head()

Row(age=58, job='management', marital='married', education='tertiary', default='no', balance=2143, housing='yes', loan='no', contact='unknown', day=5, month='may', duration=261, campaign=1, pdays=-1, previous=0, poutcome='unknown', y='no', age_group=3.0)

In [31]:
indexer = StringIndexer(inputCols=["marital", "education", "housing", "loan", "month"], outputCols=["marital_indexed", "education_indexed", "housing_indexed", "loan_indexed", "month_indexed"])

In [32]:
indexed_df = indexer.fit(df).transform(df)

In [33]:
indexed_df.select("age_group", "marital_indexed", "education_indexed", "housing_indexed", "loan_indexed", "month_indexed", "y").show()

+---------+---------------+-----------------+---------------+------------+-------------+---+
|age_group|marital_indexed|education_indexed|housing_indexed|loan_indexed|month_indexed|  y|
+---------+---------------+-----------------+---------------+------------+-------------+---+
|      3.0|            0.0|              1.0|            0.0|         0.0|          0.0| no|
|      2.0|            1.0|              0.0|            0.0|         0.0|          0.0| no|
|      1.0|            0.0|              0.0|            0.0|         1.0|          0.0| no|
|      2.0|            0.0|              3.0|            0.0|         0.0|          0.0| no|
|      1.0|            1.0|              3.0|            1.0|         0.0|          0.0| no|
|      1.0|            0.0|              1.0|            0.0|         0.0|          0.0| no|
|      0.0|            1.0|              1.0|            0.0|         1.0|          0.0| no|
|      2.0|            2.0|              1.0|            0.0|         